Gerekli kütüphaneleri yüklüyoruz.

In [1]:
import cv2
import numpy as np
from scipy.signal import find_peaks
from scipy.signal import butter, filtfilt

In [2]:
from ultralytics import YOLO
# Burada kullanacağımız modeli seçiyoruz. Bu model ile yüz tespiti yapacağız
model= YOLO("yolov8n-face.pt") 

Bandpass filtre tanımlıyoruz. Bu filtre ile belirli değerler arasındaki sinyalleri alıyoruz. Bu değer aralığının dışındaki sıfır yapıyor.

In [3]:
def temporal_bandpass_filter(data, fs, lowcut, highcut, order=1):
    
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data, axis=0)
    return y

Kodda kullanacağımız önemli fonksiyonlardan bir tanesi. Burada ilk önce resmi levels sayısı kadar boyut olarak yarıda birine düşürüyoruz. Örneğin levels=3 olursa 3 sefer yarıda bire düşürme işlemi uygularız. Böylece görüntü hem genişlik hem de yükseklik olarak 1/8 boyutuna ulaşacak. Ardından levels sayısı kadar resmi 2 katına çıkarıyoruz. Böylece resim orijinal boyutuna ulaşmış olacak. Fakat arada bir kayıp olacak. Bu kaybı alpha katsayısı ile çarpıp orijinal resme ekliyoruz.

In [4]:
def amplify_spatial_laplacian_pyramid(frame, levels=3, alpha=50):
    
    pyramid = frame.copy()
    for _ in range(levels):
        pyramid = cv2.pyrDown(pyramid)
    for _ in range(levels):
        pyramid = cv2.pyrUp(pyramid)
    laplacian = frame - pyramid
    amplified_frame = frame + alpha * laplacian
    return amplified_frame

Burada amaç tahmini olarak kalp atış hızını belirlemek. Fakat burada birçok bağlı değişken var. Bundan dolayı videodan videoya bu değerlerin kalibre edilmesi gerekebilir. Ayrıca buradaki sonuçları direk bir fiziksel cihaza bağlayıp ölçmediğim için sonuçların %100 doğru olduğunu söyleyemem. Fakat yaptığım hesaplar sonucunda bu video için buradaki değerlerin yüksek doğrulukla sonuç verdiklerini buldum. siz kendi videonuz için buradaki değerleri değiştirebilirsiniz.

Kodun çalışma mantığı ise şöyle. İnsan yüzünde çok küçük renk değişimleri olur. Bunlar gözle görülmesi çok zor. Burada bu değişimleri yakalayıp bunları büyütüp ardından bunların ortalamasını alıp bir sinyal oluşturyoruz.Bu sinyalde zirveyi temsil eden noktalar genelde nabzın attığı yeri ifade diyor. Yani renk değişiminin en fazla olduğu noktalarda nabız atışı oluyor diyebiliriz. Ardından bu noktaları kullanarak frekanstan kalp atış hızını hesaplama yapıyoruz. 

Burada ilk önce belirli sayıda frame birikmesi gerekiyor. Ardından bunları işleme sokup nabzı tahmin ediyoruz. Ben bu tahmin işleminde kullanılacak frame sayısını 150 olarak belirledim. 150. frame'den sonra hesaplama başlıyor. Sonraki tahmin işlemini ise anında değil 30 frame sonra yapsın diye belirledim. Yani her 30 frame'de bir tahmin işlemi yaplıyor. Bu da kabaca 1 saniyeye denk geliyor. Ayrıca burada sadece yeşil kanalı kullandım. Mavi ve kırmızı renkteki kanalı kullanmadım. Okuduğum makalelerde sadece yeşil rengin kullanılmasının  daha iyi sonuçlar verdiği söyleniyordu. Siz isterseniz diğer renkleri de kullanabilirsiniz

Buradaki kod videoda tek kişinin olduğu videolar için ayarlanmıştır. Videoda birden fazla kişi varsa tracking işlemi uygulanıp buradaki adaımlar her bir yüz için tekrarlanmalı

In [5]:
# Videoyu yüklüyoruz
cap= cv2.VideoCapture('video.mp4')

# Bu kodu yorumdan çıkarırsanız bilgisayarın kamerası kullanılır.
#cap= cv2.VideoCapture(0)

# Videonun FPS'sini alıyoruz.
frame_rate = cap.get(cv2.CAP_PROP_FPS)

#Yazı fontunu ayarlıyoruz.
font = cv2.FONT_HERSHEY_SIMPLEX

# Kalp atışı hızı
heart_rate=0

# Renk değşimlerinin ortalamasını bu listeye ekliyoruz.
mean_values = []

# Her 30 Frame'de bir tahmin yapıyoruz. Bunu kontrol etmek için kullanacağımız değişken
buffer_count=0


while True:
    # Görüntüyü alıyoruz videodan
    ret, frame = cap.read()
    
    # Bu değişkeni her frame'de bir arttıryoruz.
    buffer_count+=1
  
    # Kameradan görüntü gelmezse döngğden çıkar.
    if not ret:
        break
    
    # Resmi RGB uzayına çeviriyoruz.
    imgs=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    # Resme blur uyguluyoruz, gürültüyü azaltmak için
    imgs = cv2.GaussianBlur(imgs, (5, 5), 0)
    
    # Resmi yüz tespiti yapabilen modele resmi verdik.
    results = model(imgs,verbose=False) 
    
    # Modelin tespit edebildiği sınıflar. Buradaki sonuç sadace insan yüzü.
    labels=results[0].names
    
    # Her bir nesne yani yüz için bir for döngüsü çalışacak. Burada yüzün koordinatlarını buluyoruz.
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        label=results[0].boxes.cls[i]
        x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
        
        # Yüzü mavi bir dikdörtgen içine alıyoruz.
        cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),10)
        
        
        # Burada yüzün olduğu olduğu kısmı görselden kırpacağız. Görseli 3 kere 2'de bir oranı küçültüp büyüteceğimiz 
        # Yani 8 kat olduğu için resmin hem genişliği hem de hem de yüksekliği 8' tam bölünebilmeli. 
        # Diğer türlü resim boyutu uyumsuz olacağı için hata verecek küçülüp büyürken
        # O yüzden iki uç noktadan çıkarıp genişlik ve yükseliklik tanımlarken bu değerlerden 8'e bölününce kalan 
        # değerleri de çıakrıyoruz.
        h=y2-y1-((y2-y1)%8)
        w=x2-x1-((x2-x1)%8)
        
        # Görseli yüz kısmından kesitk. İsterseniz sadece alın bölgesini de kullanabilirsiniz.
        roi_color = frame[y1:y1+h, x1:x1+w]
        
        # Yüzü amplify fonksiyonuna veriyoruz.
        roi_color = amplify_spatial_laplacian_pyramid(roi_color, levels=3, alpha=200)
        
        # Resmin yeşil kanalını alıyoruz.
        green_channel = roi_color[:, :, 1]
        
        # Burada ortalama alıyoruz.
        mean_val = np.mean(green_channel)
        
        #Bunu da bu listeye ekliyoruz. Bu her bir frame için yapılıyor. Yani bu listedeki her bir değer
        # her bir frame'nin yeşil kanalındaki değişimin ortalaması
        mean_values.append(mean_val)
        
        # burada fonksiyondan gelen görüntüyü ekranda gösteriyoruz.
        cv2.imshow("roicolor",roi_color)
        # Sadece tek bir yüz alsın diye koddan çıkıyoruz.
        break
    
    # Eğer biriken ortalama sayısı 150 olmuşsan ve son tahmin işleminden itibaren 30 frame geçmişse tahmin kısmına geçiyoruz.
    
    if len(mean_values)>149 and buffer_count>29:
        
        # Bu değişkeni her tahmin işleminde sıfırlıyoruz.
        buffer_count=0 
        
        # Numpy array'e çeviriyoruz
        mean_values_array = np.array(mean_values)
        
        # Her seferinde 150 değere baktığımız için ilk 30 değeri birikme olmasın diye siliyoruz
        del mean_values[:30]
        
        # Sinyale bandpass filtre uyfuluyoruz. Buradaki 0.8 48 nabzı 2 ise 120 nabzı temsil ediyor. 
        # Bu değerleri tahmini en düşük ve en yüksek nabız değerleri
        filtered_signal = temporal_bandpass_filter(mean_values_array, frame_rate, lowcut=0.8, highcut=2)
        
        # Burada zirveleri buluyoruz. Buradaki 2.0 değeri zirveler arasındaki fark için ayarladığımız bir
        # Bu koddaki en önemli parametre. O yüzden bunu değiştirirken dikkatli değiştirin. Yoksa çok uçuk
        # değerler bulabilirsiniz.
        peaks, _ = find_peaks(filtered_signal, distance=frame_rate/2.0)
        
        # Zirve sayısı birden büyükse kod çalışacak
        if len(peaks) > 1:
            
            # burada zirveler arasındaki farkı bulup FPS'e bölüyoruz.
            intervals = np.diff(peaks) / frame_rate
            
            # Ardından bunların ortalamasını alıp 60'a bölüyoruz. 
            # Çünkü kalp atış hızı olan BPM dakikadaki kalp atış hızını söyler
            heart_rate = 60 / np.mean(intervals)
            
    # Kalp atış hızı hesaplanmışsa 0'dan büyüktür. Diğer türlü default olan 0'dır
    if heart_rate>0:
        
        # Burada ekranda iyi gözüksün diye sol üst köşeyi mor renk yapıyoruz.
        frame[0:100,0:400]=(102,0,153)
        
        # Ekranın  sol üst köşesine kalp atış hızını yazdırıyoruz.
        text='BPM:'+str(int(heart_rate))
        cv2.putText(frame, text,(30, 80), font, 3, (255,255,255), 3)
    
    cv2.imshow("kamera",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()